In [5]:
from ipywidgets import widgets, IntSlider, Label
from IPython.display import display

import matplotlib.pyplot as plt
import numpy as np
from sympy.abc import tau
from sympy import latex
import math

In [6]:
def evolution_of_carbon_mantle(t, T_am, M_cmp, M_cccp, M_ca0):
    M_cm = M_cmp + M_cccp - M_ca0 + (M_ca0 - M_cccp)*(1 - np.exp(-t/T_am))
    return M_cm

def evolution_ccc_initial(t,t_cc0):
    M_ccc_initial = np.zeros(t[t<t_cc0].size)
    return M_ccc_initial

def evolution_of_carbon_continental_crust(t, t_cc0, T_acc, M_cccp):
    M_ccc = M_cccp * (1-np.exp(-1 * (t[t>=t_cc0]-t_cc0)/(T_acc)))
    return M_ccc

def evolution_of_carbon_atmosphere_initial(t, t_cc0, T_ca, M_ca0, M_cccp):
    M_ca_initial = M_ca0 - (M_ca0 - M_cccp)*(1 - np.exp(-t[t<t_cc0]/T_ca))
    return M_ca_initial

def evolution_of_carbon_atmosphere(t, t_cc0, T_acc, T_am, M_cmp, M_cccp, M_ca0):
    M_ca = (M_ca0 - M_cccp)*(np.exp(-t[t>=t_cc0]/T_am)) + M_cccp * np.exp(-1 * (t[t>=t_cc0]-t_cc0)/(T_acc))
    #M_ca = M_cmp - evolution_of_carbon_mantle(t[t>=t_cc0],T_am, M_cmp, M_cccp, M_ca0) + M_cccp - evolution_of_carbon_continental_crust(t[t>=t_cc0], t_cc0, T_acc, M_cccp)
    return M_ca

def time(tp):
    return np.arange(0,tp,1e6)

billion_years = 1e9
gigaton = 1e8
t = time(4.4e9)

In [7]:
def reservoir_model(t, t_cc0, T_am, T_acc, M_cmp, M_cccp, M_ca0):
    plt.close('all')
    plt.figure(figsize=(8,6))
    M_cm = evolution_of_carbon_mantle(t, T_am, M_cmp, M_cccp, M_ca0)
    M_cm_line = plt.plot(t/billion_years, M_cm/gigaton)
    plt.setp(M_cm_line, color='orange')
    plt.annotate('Mantle', xy=(0.05, 0.95), xytext=(t[-1]*0.75/billion_years, M_cm[-1]*0.92/gigaton))
    
    M_ccc_initial = evolution_ccc_initial(t, t_cc0)
    M_ccc = np.append(M_ccc_initial, evolution_of_carbon_continental_crust(t, t_cc0, T_acc, M_cccp))
    M_ccc_line = plt.plot(t/billion_years, M_ccc/gigaton)
    plt.setp(M_ccc_line, color='black')
    plt.annotate('Continental Crust', xy=(0.05, 0.95), xytext=(t[-1]*0.65/billion_years, M_ccc[-1]*1.20/gigaton))

    
    M_ca_initial = evolution_of_carbon_atmosphere_initial(t, t_cc0, T_am, M_ca0, M_cccp)
    M_ca = np.append(M_ca_initial, evolution_of_carbon_atmosphere(t, t_cc0, T_acc, T_am, M_cmp, M_cccp, M_ca0))
    M_ca_line = plt.plot(t/billion_years, M_ca/gigaton)
    plt.setp(M_ca_line, color='cyan')
    plt.annotate('Atmosphere', xy=(0.05, 0.95), xytext=(t[-1]*0.70/billion_years,(M_ca[-1] + 0.1e8)/gigaton))

    plt.xlabel('Time ($10^9$ years)')
    plt.ylabel('Mass of Carbon ($10^8$ Gt)')
    #plt.savefig('model_2_plot.pdf', format='pdf')
    plt.show()

# Reservoir Model of Evolution of Deep Carbon

## Model 1

In [8]:


M_ca0_widget = widgets.IntSlider(min=3e7,max=2e8,step=1e7,value=1.57e8, description="$M_{ca0}$", readout_format=".2g")
M_cmp_widget = widgets.IntSlider(min=1e8,max=3e8,step=1e7,value=2e8, description="$M_{cmp}$", readout_format=".2g")
M_cccp_widget = widgets.IntSlider(min=0,max=6e7,step=1e6,value=4.2e7, description="$M_{cccp}$", readout_format=".2g")
t_cc0_widget = widgets.IntSlider(min=0.5e9,max=2e9,step=1e8,value=0.5e9, description="$t_{cc0}$", readout_format=".2g")
T_am_widget = widgets.IntSlider(min=1e7,max=1e9,step=1e7,value=2e7, description="$"+latex(tau)+"_{am}$", readout_format=".2g")
T_acc_widget = widgets.IntSlider(min=0.5e9,max=2e9,step=1e8,value=1e9, description="$"+latex(tau)+"_{acc}$", readout_format=".2g")

w = widgets.interactive(reservoir_model,
                        t = widgets.fixed(t),
                        t_cc0 = t_cc0_widget,
                        T_am = T_am_widget,                        
                        T_acc = T_acc_widget,
                        M_cmp = M_cmp_widget,
                        M_cccp = M_cccp_widget,
                        M_ca0 = M_ca0_widget)

display(w)

interactive(children=(IntSlider(value=500000000, description='$t_{cc0}$', max=2000000000, min=500000000, readout_format='.2g', step=100000000), IntSlider(value=20000000, description='$\\tau_{am}$', max=1000000000, min=10000000, readout_format='.2g', step=10000000), IntSlider(value=1000000000, description='$\\tau_{acc}$', max=2000000000, min=500000000, readout_format='.2g', step=100000000), IntSlider(value=200000000, description='$M_{cmp}$', max=300000000, min=100000000, readout_format='.2g', step=10000000), IntSlider(value=42000000, description='$M_{cccp}$', max=60000000, readout_format='.2g', step=1000000), IntSlider(value=157000000, description='$M_{ca0}$', max=200000000, min=30000000, readout_format='.2g', step=10000000), Output()), _dom_classes=('widget-interact',))